In [213]:
import pandas as pd
import dash
import more_itertools
import dash_bootstrap_components as dbc
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go

In [214]:
# Load the Dataset
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/d51iMGfp_t0QpO30Lym-dw/automobile-sales.csv')

In [216]:
# Initialize the Dash App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])

# Set the title
app.title = 'Automobile Analysis'

In [217]:
# Helper function to style charts (Makes them look clean & transparent)
def style_fig(figure):
    figure.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',       # Transparent background
        paper_bgcolor='rgba(0,0,0,0)',      # Transparent container
        font_family="Segoe UI, sans-serif", # Modern Font
        title_font_size = 18,
        title_x= 0.5,                       # Center the title
        margin=dict(l=20, r=20, t=40, b=20) # Clean margins
    )
    # Add a nice grid for readability
    figure.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#E5E5E5')
    figure.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#E5E5E5')
    
    return figure

In [218]:
# Create the layout
app.layout = dbc.Container([

    # ---Header---
    dbc.NavbarSimple(
        brand="Automobile Sales Analysis Dashboard",
        brand_href="#",
        color="primary",    # Uses the theme's primary color (Blue/Purple)
        dark=True,          # White text
        className="mb-4 shadow-sm",
        brand_style={
            'position' : 'absolute',
            'left' : '50%',
            'transform': 'translateX(-50%)',
            'width': 'auto' # Ensure it doesn't try to stretch
        }
    ),

    # --- Controls Section (Inside a Card) ---
    dbc.Card([
        dbc.CardBody([
            dbc.Row([
                # Statistics Dropdown
                dbc.Col([
                    html.Label("Select Report Type:", className="fw-bold text-muted"),
                    dcc.Dropdown(
                        id='dropdown-statistics',
                        options=[
                            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                        ],
                        value='Select Statistics',
                        placeholder="Select a report type",
                        clearable=False,
                        style={'fontsize' : '16px'}
                    )
                ], width=12, md=6), # width=12 on mobile, 6 on desktop

                # Year Dropdown
                dbc.Col([
                    html.Label("Select Year:", className="fw-bold text-muted"),
                    dcc.Dropdown(
                        id='select-year',
                        options = [{'label': i, 'value': i} for i in range(1980, 2024, 1)],
                        value='Select Year',
                        placeholder='Select Year',
                        disabled=True, # Disabled by Default until needed
                        style={'fontsize': '16px'}
                    )
                ], width=12, md=6)
            ])
        ])
    ], className="mb-4 shadow-sm"),

    # --- Output Section ---
    html.Div(id='output-container')
], fluid=True, style={'backgroundColor': "#f8f9fa", 'minHeight': '100vh'}) # Light grey page background


In [219]:
# Create the Callback function to update the input container based on the selectetd statistics
@app.callback(
    Output(component_id='select-year',component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)

# This function will enable the 'Select Year' dropdown if users select 'Yearly Statistics' input, vice versa.
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

In [220]:
# Callback for Plotting
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)

def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]

    # Create and Display graphs for Recession Period Statistics
        # Plot 1: Automobile sales fluctuate over Recession Period (year wise)
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = px.line(yearly_rec, x='Year', y='Automobile_Sales', markers=True,
                       title="Average Sales Fluctuation (Recession)")

        # Plot 2: Calculate the average number of vehicles sold by vehicle type
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig2 = px.bar(average_sales, x='Vehicle_Type', y='Automobile_Sales', color='Vehicle_Type',
                       title="Average Sales Fluctuation (Recession)")

        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig3 = px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type',
                      title='Advertisement Expenditure Share', hole=0.4) # Hole make it a donut

        # Plot 4: Bar chart for the effect of unemployment rate on vehicle type and sales
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'], as_index=False)['Automobile_Sales'].mean().reset_index()
        fig4 = px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                      title="Effect of Unemployment Rate on Sales")

        # App style & return layout
        return [
            dbc.Row([
                dbc.Col(dcc.Graph(figure=style_fig(fig1)), width=12, lg=6),
                dbc.Col(dcc.Graph(figure=style_fig(fig2)), width=12, lg=6)
            ], className="mb-4"),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=style_fig(fig3)), width=12, lg=6),
                dbc.Col(dcc.Graph(figure=style_fig(fig4)), width=12, lg=6)
            ])
        ]
    
    # Create and Display Graphs for Yearly Statistics
    elif (input_year and selected_statistics == 'Yearly Statistics'):
        
        # SAFETY CHECK: If the user hasn't picked a year yet, stop here to avoid error
        if input_year == 'Select Year':
            return None
        
        yearly_data = data[data['Year'] == input_year]

        # Plot 1: Yearly Automobile Sales using line chart for the whole period
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = px.line(yas, x='Year', y='Automobile_Sales', markers=True,
                       title="Yearly Automobile Sales Trend")

        # Plot 2: Total Monthly Automobile Sales using line chart
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        mas['Month'] = pd.Categorical(mas['Month'], categories=order, ordered=True)
        mas = mas.sort_values('Month')
        fig2 = px.line(mas, x='Month', y='Automobile_Sales', markers=True,
                       title="Total Monthly Sales")

        # Plot 3: Bar chart for average number of vehicles sold during the given year
        avg_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig3 = px.bar(avg_vdata, x='Vehicle_Type', y='Automobile_Sales', color='Vehicle_Type',
                      title=f"Average Vehicles Sold by Type in {input_year}")

        # Plot 4: Total Advertisement Expenditure for each vehicle type using pie chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig4 = px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type',
                      title=f"Advertisement Expenditure by Type in {input_year}", hole=0.4)
        
        # Apply Style & Return Layout
        return [
            dbc.Row([
                dbc.Col(dcc.Graph(figure=style_fig(fig1)), width=12, lg=6),
                dbc.Col(dcc.Graph(figure=style_fig(fig2)), width=12, lg=6)
            ], className="mb-4"),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=style_fig(fig3)), width=12, lg=6),
                dbc.Col(dcc.Graph(figure=style_fig(fig4)), width=12, lg=6)
            ])
        ]
    
    else:
        return None

In [221]:
# Run the Dash App
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
